<a href="https://colab.research.google.com/github/softmancho/Meeting-Recording-Summarisation-MVP-Internship-Project/blob/justice/meeting_summarisation_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#wrap the output in colab cells
import IPython
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre{
      white-space: pre-wrap;
    }
  
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [24]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
file = open('/content/drive/MyDrive/meeting_recording_to_text.txt', 'r')

In [28]:
FileContent = file.read().strip()

In [29]:
FileContent

'Das : Hi and welcome to the a16z podcast. I’m Das, and in this episode, I talk SaaS go-to-market with David Ulevitch and our newest enterprise general partner Kristina Shen. The first half of the podcast looks at how remote work impacts the SaaS go-to-market and what the smartest founders are doing to survive the current crisis. The second half covers pricing approaches and strategy, including how to think about free versus paid trials and navigating the transition to larger accounts. But we start with why it’s easier to move upmarket than down… and the advantage that gives a SaaS startup against incumbents.\nDavid : If you have a cohort of customers that are paying you $10,000 a year for your product, you’re going to find a customer that self-selects and is willing to pay $100,000 a year. Once you get one of those, your organization will figure out how you sell to, how you satisfy and support, customers at that price point and that size. But it’s really hard for a company that sells 

In [30]:
len(FileContent)

23640

In [31]:
FileContent

'Das : Hi and welcome to the a16z podcast. I’m Das, and in this episode, I talk SaaS go-to-market with David Ulevitch and our newest enterprise general partner Kristina Shen. The first half of the podcast looks at how remote work impacts the SaaS go-to-market and what the smartest founders are doing to survive the current crisis. The second half covers pricing approaches and strategy, including how to think about free versus paid trials and navigating the transition to larger accounts. But we start with why it’s easier to move upmarket than down… and the advantage that gives a SaaS startup against incumbents.\nDavid : If you have a cohort of customers that are paying you $10,000 a year for your product, you’re going to find a customer that self-selects and is willing to pay $100,000 a year. Once you get one of those, your organization will figure out how you sell to, how you satisfy and support, customers at that price point and that size. But it’s really hard for a company that sells 

In [32]:
len(FileContent.split())

4255

In [33]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [34]:
def meeting_summary(checkpoint='t5-base',
                    checkpoint_tokenizer=T5Tokenizer, 
                    checkpoint_model=T5ForConditionalGeneration,
                    meeting_recording_to_text =FileContent):
  
  checkpoint = checkpoint
  model = checkpoint_model.from_pretrained(checkpoint)
  tokenizer = checkpoint_tokenizer.from_pretrained(checkpoint)
  device = torch.device('cuda')

In [36]:
# initialize the pretrained model
checkpoint ='t5-base'
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5Tokenizer.from_pretrained(checkpoint)
device = torch.device('cuda')

# Transformer model statistics  .
#### First we want to check some statistics of our model, we want to check the number of tokens our model can support including the special tokens 

In [39]:
tokenizer.model_max_length

512

####  Next we want to check the number of tokens our model can support excluding the special tokens

In [40]:
tokenizer.max_len_single_sentence

511

#### So from this result we can see that the model can adds one special token for the input sequence. which we can verify from the nest code

In [41]:
tokenizer.num_special_tokens_to_add()

1

# Meeting_recording_to_text Statistics

#### Next we have to covert the file content that has been loaded from google drive into sentenses, doing this we need an external library call NLTK

In [42]:
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#### we want to know how many sentences are contained in our file

In [43]:
len(sentences)

199

#### we want to know the maximum tokens in the longest sentence in our file. and from our code below we can observe that the maximum token from the longest sentence in our file which is 97, is way below the max number of tokens our model can handle or support which is 511

In [44]:
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

97

#### Next we will create sentence chunks by combining the sentences such that each chunk of sentences contans less than or equal to the maximum  number of token allowed by the model.

In [45]:
from nltk.translate.chrf_score import sentence_chrf
from nltk.tokenize.treebank import TokenizerI
lenght=0
chunk =""
chunks = []
count = -1

#loop through the entire sentence in the file
for sentence in sentences:
  #increment a counter
  count+=1
  #then find out the conmined lengght of the sentence
  combined_lenght = len(tokenizer.tokenize(sentence)) + lenght

  #next check if the combined lenght is below the limit of the maximum number of token our model can handle
  if combined_lenght <= tokenizer.max_len_single_sentence:
    #add the sentence to the chunk along with a white-space after the sentence
    chunk += sentence + ' '
    #then we need to update the lenght to combined lenght
    lenght = combined_lenght

    #let us check wether its the last sentence of the file
    if count == len(sentences) - 1:
      #if it is so the we need to strip the chunk and save it
      chunks.append(chunk.strip())

  #if the combined_lenght reaches the max number of token our model can handle,
  else:
    #then we will save the chunk first, 
    chunks.append(chunk.strip())
    # and then reset the lenght and chunk
    lenght = 0
    chunk = ''

    #we will also need to take care of the overflow  sentence which make the count go higher than
    #the allowable lenght
    #first we update the sentence,
    chunk += sentence + " "
    # and then we update the lenght
    lenght = len(tokenizer.tokenize(sentence))

#lets print the combined sentences
len(chunks)







12

#### We have created 12 chunks from our file,  Lets find out the number of tokens excluding the special tokens in each of the 12 chunks

In [47]:
[len(tokenizer.tokenize(c)) for c in chunks]

[490, 508, 507, 501, 508, 499, 498, 505, 492, 507, 478, 156]

#### Let us also check the number of tokens including the special tokens in our file

In [48]:
[len(tokenizer(c).input_ids) for c in chunks]

[491, 509, 508, 502, 509, 500, 499, 506, 493, 508, 479, 157]

#### Next we want to find out the total number of tokens in all the chunk, which should be seen as equal to the total mmunber of token in the original file content. 

In [49]:
sum([len(tokenizer.tokenize(c))for c in chunks])

5649

In [50]:
len(tokenizer.tokenize(FileContent))

5649

#Modelling

#### next we create the input tensors for our model

In [59]:
inputs = [tokenizer.encode(chunk,return_tensors='pt') for chunk in chunks]

In [60]:
inputs[0]

tensor([[  644,     3,    10,  2018,    11,  2222,    12,     8,     3,     9,
          2938,   172,  8661,     5,    27,    22,    51,   644,     6,    11,
            16,    48,  5640,     6,    27,  1350,  1138,     9,   134,   281,
            18,   235,    18,  8809,    28,  1955,   412, 10912,  7059,    11,
            69,     3, 10909,  5399,   879,  2397,  9375,    17,    77,     9,
           451,    29,     5,    37,   166,   985,    13,     8,  8661,  1416,
            44,   149,  4322,   161, 11737,     8,  1138,     9,   134,   281,
            18,   235,    18,  8809,    11,   125,     8,  2592,   222,  7174,
             7,    33,   692,    12,  7905,     8,   750,  5362,     5,    37,
           511,   985,  3792,  5769,  6315,    11,  1998,     6,   379,   149,
            12,   317,    81,   339,     3,  8911,  1866, 10570,    11,     3,
         27016,     8,  3508,    12,  2186,  3744,     5,   299,    62,   456,
            28,   572,    34,    22,     7,  1842,  

# summary
#### Next we create our outputs tensors and decode them to summary

*   List item
*   List item



In [62]:
for input in inputs:
  output = model.generate(input, min_length=10, max_length=30)
  summary = tokenizer.decode(output[0], skip_special_tokens=True)
  print(summary)


we’re talking about the SaaS go-to-market with David Ulevitch and Kristina Shen. David
it was just, “Hey, we’ve got a great freemium motion going.” And then they were all bottoms
they’re doing deals on the inside, without having to engage with field sales teams. And that’s a huge catalyst from people
people started to realize they had the power, they wanted to use better software, they paid with their credit cards. that was the catalyst
you’re paying for every single seat. you’re paying for every single seat. it’s much easier to be 10
it’s very aligned with your productivity. based on your productivity. based on your productivity. you can
they’ll have a third package where you have to contact sales to find out more. And that gives the salespeople the flexibility to
you should also pay attention to how many deals are closing faster. a lot of people are trying to get in there and fiddle
it’s great to get more dollars out of your customer base, but treat your early customers well. if you’

In [ ]:
tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device)

Summarize Text

In [ ]:
summarizer(text)